In [28]:
import cv2
from cv2 import VideoWriter, VideoWriter_fourcc
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib as mpl
from scipy.sparse import csc_matrix, lil_matrix
from tqdm import tqdm

In [29]:
def get_bgr_img(array):
    fig = Figure(figsize=(16,10))
    canvas = FigureCanvas(fig)
    subplot = fig.subplots()
    fig.gca().invert_yaxis()
    norm = mpl.colors.Normalize(vmin=0, vmax=1)
    cbar = fig.colorbar(subplot.imshow(array), ticks=np.linspace(0,1,6),extend='both')
#     cbar = fig.colorbar.ColorbarBase(cmap=cmap, norm=norm)
#     cbar.minorticks_on()
    fig.canvas.draw()
    width, height = fig.get_size_inches() * fig.get_dpi()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)
    return cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

In [30]:
def in_house(x, y):
    def fi(hx, hy, x, y):
        return (hx < x and x < hx + 18 and hy < y and y < hy + 18)
    def se(hx, hy, x, y):
        return (hx < x < hx + 18 and hy < y < hy + 78) or (hx < x < hx + 30 and hy < y < hy + 18)
    def th(hx, hy, x, y):
        return (hx < x < hx + 18 and hy < y < hy + 78) or (hx - 12 < x < hx + 18 and hy + 60 < y < hy + 78)
    predicates = {'1' : fi, '2' : se, '3' : th}
    type_houses = {'1' : [(261, 165), (69, 111), (192, 111), (138, 165), (138, 12), (69, 264), (192, 264), (261, 12)],
               '2': [(192, 165), (192, 12), (69, 165), (69, 12)],
               '3': [(138, 204), (261, 51), (261, 204), (138, 51)]}
    for k in predicates.keys():
        for hx, hy in type_houses[k]:
            if predicates[k](hx, hy, x, y):
                return True
    return False

In [31]:
def is_edge_condition(x, y, M):
    k = 300 / M
    nx = x * k
    ny = y * k
    return in_house(nx, ny) or x > M - 1 or y > M - 1 or y < 0 or x < 0

In [32]:
def solve(M, eps, iters, output_iter=None):  
    k = 0.5
    l1 = 1
    l2 = 0
    tau = 1 / (4 * k * M * M)
    h = 1 / M
    coeffs = [
            1 - 4 * tau * k / (h**2),
            tau * (k / (h**2) - l1 / (2 * h)),
            tau * (k / (h**2) + l1 / (2 * h)),
            tau * (k / (h**2) - l2 / (2 * h)),
            tau * (k / (h**2) + l2 / (2 * h))
        ]
    dx = [0, 1, -1, 0, 0]
    dy = [0, 0, 0, 1, -1]
    
    A = lil_matrix((M*M, M*M), dtype=np.float64)
    b = np.zeros(M*M)
    
    for x in range(M): 
        for y in range(M):
            i = x * M + y
            A[i, i] = coeffs[0]
            if edge_condition(x, y, M):
                continue
            for d in range(1, 5):
                nx = x + dx[d]
                ny = y + dy[d]
                if nx == 0:
                    b[i] += coeffs[d]
                elif is_edge_condition(nx, ny, M):
                    A[i, i] += coeffs[d]
                else:
                    A[i, nx * M + ny] = coeffs[d]

    A = csc_matrix(A)
    u = np.zeros(M * M)
    u[0:M] = 1
    
    if output_iter is not None:
        img = get_bgr_img(u.reshape((M, M)).T)
        width, height = img.shape[:-1]
        vid = VideoWriter("diffusion.avi", VideoWriter_fourcc(*"XVID"), 10.0, (height, width))
    
    for i in tqdm(range(iters)):
        u_new = A * u + b
        error = np.max(np.abs((u_new - u) / np.maximum(1, u)))
        u = u_new
        if output_iter is not None and i % output_iter == 0:
            vid.write(get_bgr_img(u.reshape((M, M)).T))
        if error < eps:
            print(f'error less than {eps}; break')
            break
    if vid is not None:
        vid.release()
    return u.reshape((M, M)).T

In [ ]:
res = solve(M = 200, eps=1e-5, iters=100500, output_iter=100)

 34%|███▍      | 34178/100500 [01:21<02:09, 511.55it/s]